# Image classification with Convolutional Neural Networks
---

# Import Libraries

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

# Global Config

In [3]:
PATH = "../data/dogscats/"
sz=224
arch=vgg16
bs=32

In [4]:
# Uncomment the below if you need to reset your precomputed activations
!rm -rf {PATH}tmp

# Training

## 1. Train last layer for 3 cycles with Precompute

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz), bs=bs)

In [6]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

 15%|█▌        | 96/625 [05:45<31:45,  3.60s/it]

KeyboardInterrupt: 

In [10]:
learn.fit(0.01, 3, cycle_len=1)

A Jupyter Widget

[ 0.       0.05266  0.02898  0.99072]                         
[ 1.       0.04534  0.02512  0.99023]                         
[ 2.       0.04645  0.02377  0.99121]                         



## 2. Data Augmentation with 

In [4]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [6]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.05622  0.0268   0.98926]                         
[ 1.       0.04222  0.02508  0.99072]                          



In [7]:
learn.precompute=False

In [8]:
learn.fit(1e-2, 1, cycle_len=1)

A Jupyter Widget

[ 0.       0.05511  0.02654  0.98909]                         



In [8]:
learn.unfreeze()

In [9]:
lr=np.array([1e-4,1e-3,1e-2])

In [10]:
learn.fit(lr, 1, cycle_len=1)

A Jupyter Widget

[ 0.       0.05664  0.0245   0.99023]                         



In [11]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.05955  0.02217  0.9917 ]                         
[ 1.       0.03902  0.0181   0.99414]                         
[ 2.       0.03676  0.0168   0.99414]                         
[ 3.       0.03097  0.01607  0.99316]                         
[ 4.       0.02823  0.01574  0.99316]                         
[ 5.       0.02663  0.01448  0.99316]                         
[ 6.       0.03113  0.01642  0.99414]                         



In [14]:
learn.fit(lr, 3, cycle_len=3)

A Jupyter Widget

[ 0.       0.02769  0.01896  0.99268]                         
[ 1.       0.01939  0.01417  0.99365]                         
[ 2.       0.01935  0.0142   0.99414]                         
[ 3.       0.02465  0.01641  0.99316]                         
[ 4.       0.02753  0.01376  0.99365]                         
[ 5.       0.01721  0.01413  0.99414]                         
[ 6.       0.0221   0.01317  0.99365]                         
[ 7.       0.01789  0.01379  0.99268]                         
[ 8.       0.01861  0.0139   0.99316]                         



In [15]:
log_preds,y = learn.TTA()
accuracy(log_preds,y)

0.995